In [44]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [45]:
client.search_experiments()

[<Experiment: artifact_location='file:///C:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/code/mlruns/5', creation_time=1716965935679, experiment_id='5', last_update_time=1716965935679, lifecycle_stage='active', name='nyc-taxis', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/code/mlruns/3', creation_time=1716532631531, experiment_id='3', last_update_time=1716532631531, lifecycle_stage='active', name='nyc-taxi', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/code/mlruns/0', creation_time=1716532340085, experiment_id='0', last_update_time=1716532340085, lifecycle_stage='active', name='Default', tags={}>]

In [46]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='3',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]  
)

In [47]:
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics}') 

run id: a659b685bcb64c08be9d0af6fc9020ce, rmse: {'rmse': 6.313315796028479}
run id: 0f6aa7c7fedc423584dabd6b3d656280, rmse: {'rmse': 6.313315796028479}
run id: d80f9b765a7a4a848036666e43afe575, rmse: {'rmse': 6.313315796028479}
run id: b9400029a4d243f598abda6018dce305, rmse: {'rmse': 6.313315796028479}
run id: 2c5d833648c8454ab3191fe2a920caeb, rmse: {'rmse': 6.313315796028479}


In [48]:
import mlflow

mlflow.set_tracking_uri=(MLFLOW_TRACKING_URI)

In [61]:
run_id = "e57bd274605443459c17ed734c9fdaa5"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name="nyc-regressor")

Registered model 'nyc-regressor' already exists. Creating a new version of this model...


MissingConfigException: Yaml file 'C:\Users\crab\py2\mlops-zoomcamp\cohorts\2024\02-experiment-tracking\code\mlruns\3\e57bd274605443459c17ed734c9fdaa5\meta.yaml' does not exist.

In [57]:
# create "champion" alias for version 1 of model "example-model"
from mlflow import MlflowClient

client = MlflowClient()
client.set_registered_model_alias("nyc-regressor", "champion", 5)

In [58]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias,  X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [60]:
name = "nyc-regressor"
alias = "champion"


model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
y_pred = model.predict(X_test)

OSError: No such file or directory: 'runs\artifacts\e57bd274605443459c17ed734c9fdaa5\model\.'

In [53]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [54]:
client.download_artifacts(run_id='e57bd274605443459c17ed734c9fdaa5', path='preprocessor', dst_path='.')

MissingConfigException: Yaml file 'C:\Users\crab\py2\mlops-zoomcamp\cohorts\2024\02-experiment-tracking\code\mlruns\3\e57bd274605443459c17ed734c9fdaa5\meta.yaml' does not exist.

In [14]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
X_test = preprocess(df,dv)

In [16]:
target = "duration"
y_test = df[target].values

In [59]:
model_name = 'nyc-regressor'

alias = 'Champion'
  
%time test_model(name=model_name, alias=alias, X_test=X_test, y_test=y_test) 

nyc-regressor@Champion


OSError: No such file or directory: 'runs\artifacts\e57bd274605443459c17ed734c9fdaa5\model\.'